In [ ]:
import tensorflow as tf
from tf.keras.applications import ResNet50
from tf.keras.models import Model, Sequential
from tf.keras.layers import Dense, GlobalAveragePooling2D, Embedding, LSTM
from tf.keras.losses import CategoricalCrossEntropy
from tf.keras.optimizers import Adam


import numpy as np


In [ ]:
input_shape = (224, 224, 3)
num_epochs = 50 # not sure tbh

# **Building the model:**
## CNN: ResNet50

In [ ]:
# Loading the pretrained ResNet on ImageNet dataset
resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape= input_shape)

# Freezing the layers
for layer in resnet_model.layers:
    layer.trainable = False
    
features = resnet_model.output

FC_layer = GlobalAveragePooling2D()(features)
FC_layer = Dense(256, activation = 'relu')(FC_layer)

CNN_model = Model(inputs = resnet_model.input, output = FC_layer)

CNN_model.summary()

## RNN: LSTM

In [ ]:
LSTM_model = Sequential()

# Rana's part returns ----> vocabulary_size, embedding_dim, max_sequence_length
LSTM_model.add(Embedding(vocabulary_size, embedding_dim, input_length = max_sequence_length))
LSTM_model.add(LSTM(256))
LSTM_model.add(Dense(vocabulary_size, activation = 'softmax'))

In [ ]:
loss_function = CategoricalCrossEntropy()
optimizer = Adam()

LSTM_model.compile(loss = loss_function, optimizer = optimizer)
LSTM_model.summary()

In [ ]:
# Applying teacher forcing 
# Note: This part is totally predicted to not perform as demanded yet
# since the data preprocessing and captions are not available yet ==> RANAAAAAAAA

# From Rana's part ---> input_sequences, target_sequences, to_one_hot_encoding(function)
for epoch in range(num_epochs):
    for i in range(len(input_sequences)):
        hidden_state = LSTM_model.reset_states()
        
        for t in range(len(input_sequences[i])):
            input_word = input_sequences[i][t]
            target_word = target_sequences[i][t]
            
            input_word_encoded = to_one_hot_encoding(input_word, vocabulary_size)
            
            input_word_reshaped = input_word_encoded.reshape(1, 1, vocabulary_size)
            
            LSTM_output = LSTM_model(input_word_reshaped)
            
            loss = loss_function(target_word, LSTM_output)
            
            LSTM_model.train_on_batch(input_word_reshaped, target_word)